In [ ]:
from lib.models.torch_models import single_layer_perceptron, TorchRegressor
from lib.models.encdec import get_data_loaders
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.externals import joblib
from sklearn.pipeline import make_pipeline

from torch.autograd import Variable
import torch
from torch import nn

import numpy as np

In [ ]:
train_loader, test_loader = get_data_loaders("../data/ml/ngaqua/data.pkl")

In [ ]:


data = joblib.load("../data/ml/ngaqua/data.pkl")

# load weight data
w = data['w'][1]/data['scale'][1]**2
w_torch = Variable(torch.FloatTensor(w))

# mass weighted loss
mse = nn.MSELoss()

def loss_function(output, y):
    return torch.mean(torch.pow(output-y, 2).mul(w_torch))


net = TorchRegressor(single_layer_perceptron, loss_function, num_epochs=5,
                    optim_kwargs={'lr':.001})
mod = make_pipeline(VarianceThreshold(.001), StandardScaler(), net)


In [ ]:
xtrain, ytrain = map(np.asarray, data['train'])
mod.fit(xtrain, ytrain)

In [ ]:
from lib.models import weighted_r2_score

In [ ]:
x_test, y_test = data['test']

pred = mod.predict(x_test)

weighted_r2_score(y_test, pred, weight=w)

It seems like keras tends to give better performance? Why?